In [1]:
import sys
import h5py
from pathlib import Path
import zipfile, os, sys
import numpy as np
from collections import defaultdict
import re
import json
import pandas as pd

ROOT = Path.cwd().parent          
SRC  = ROOT / "src"
sys.path.insert(0, str(SRC))      

from fashiongen_h5_loader import (
    ensure_h5_ready, build_vocab_from_h5, make_loaders_with_existing_val
)

# 1. Import Data

In [2]:
DATA_DIR = ROOT / "data"
assert DATA_DIR.exists(), f"Missing data dir: {DATA_DIR}"
print("DATA_DIR:", DATA_DIR)

DATA_DIR: /Users/jiaqingzhang/Downloads/GT_DL/project/Shop-Till-You-Drop/data


In [3]:
# 1) Unzip both train/validation if needed; search h5 recursively
h5s = []
for zp in sorted(DATA_DIR.glob("*.zip")):
    out_dir = DATA_DIR / zp.stem
    out_dir.mkdir(parents=True, exist_ok=True)
    # extract only missing files
    with zipfile.ZipFile(zp, "r") as zf:
        for m in zf.infolist():
            # skip macOS metadata
            if m.filename.startswith("__MACOSX/"): 
                continue
            target = out_dir / m.filename
            if not target.exists():
                zf.extract(m, out_dir)
    # collect any .h5 inside (recursively)
    h5s.extend(sorted(out_dir.rglob("*.h5")))

In [4]:
h5s = sorted(set(h5s))
print("Found H5 files:")
for p in h5s: print(" -", p)

Found H5 files:
 - /Users/jiaqingzhang/Downloads/GT_DL/project/Shop-Till-You-Drop/data/fashiongen_256_256_train.h5/fashiongen_256_256_train.h5
 - /Users/jiaqingzhang/Downloads/GT_DL/project/Shop-Till-You-Drop/data/fashiongen_256_256_validation.h5/fashiongen_256_256_validation.h5


In [5]:
# 2) Identify splits by filename
lower = [p.name.lower() for p in h5s]
train_h5 = next((str(p) for p in h5s if "train" in p.name.lower()), None)
val_h5   = next((str(p) for p in h5s if "val" in p.name.lower() or "validation" in p.name.lower()), None)
test_h5  = next((str(p) for p in h5s if "test" in p.name.lower()), None)
print("train_h5:", train_h5)
print("val_h5  :", val_h5)
print("test_h5 :", test_h5)


train_h5: /Users/jiaqingzhang/Downloads/GT_DL/project/Shop-Till-You-Drop/data/fashiongen_256_256_train.h5/fashiongen_256_256_train.h5
val_h5  : /Users/jiaqingzhang/Downloads/GT_DL/project/Shop-Till-You-Drop/data/fashiongen_256_256_validation.h5/fashiongen_256_256_validation.h5
test_h5 : None


In [6]:
# 3) Build label vocab from train 
vocab_path = ROOT / "data/label_vocab.json"
_ = build_vocab_from_h5(
    train_h5,
    label_key="input_category",  
    add_other=True,
    min_count=5,
    out_json=str(vocab_path)
)

[vocab] Saved 49 labels to /Users/jiaqingzhang/Downloads/GT_DL/project/Shop-Till-You-Drop/data/label_vocab.json


In [7]:
# 4) Build datasets/loaders
train_loader, val_loader, test_loader, train_ds, val_ds, test_ds = make_loaders_with_existing_val(
    train_h5=train_h5,
    val_h5=val_h5,
    vocab_json=str(vocab_path),
    image_key="input_image",
    label_key="input_category",
    train_ratio=0.8,
    batch_size=64,
    num_workers=8,
)

In [8]:
print("image_key:", train_ds.image_key)
print("label_key:", train_ds.label_key)
print("caption_key:", train_ds.caption_key)

image_key: input_image
label_key: input_category
caption_key: input_description


In [9]:
with h5py.File(train_h5, "r") as f:
    for k in [train_ds.image_key, train_ds.label_key, train_ds.caption_key]:
        if k is None: 
            continue
        d = f[k]
        print(f"{k}: shape={getattr(d,'shape',None)}, dtype={getattr(d,'dtype',None)}")

input_image: shape=(260490, 256, 256, 3), dtype=uint8
input_category: shape=(260490, 1), dtype=|S100
input_description: shape=(260490, 1), dtype=|S400


In [10]:
b = next(iter(train_loader))
print(b["image"].shape, b["label"].shape)

torch.Size([64, 3, 256, 256]) torch.Size([64])
